# Mice Brain Mri Segmentation #
(actually, it can segment human scans as well. the repo name is a bit underthought)

## Overview ##
* These scripts allow to finenly segment raster images (MRI scans) using svg images (from histological brain atlas).
* Atlas svg images and some other assets are retrieved from Allen Brain Institute web-service, which is free for academic and personal use.
    * [Allen Brain Institute website](https://portal.brain-map.org/)
    * [Mouse | Human brain atlases](http://atlas.brain-map.org/)
    * [Allen Brain Institute API help](http://help.brain-map.org/display/api/Allen+Brain+Atlas+API)
* This repository contains a collection of cohesive scripts, **not a Python package or an application**. One should know Python to use this.

---

This notebook will guide through one through the path to replicate our work:
1. [installing nesessary programs](#1.-Installations)
1. [creating folders](#2a.-creating-folders) and nessesary files
1. [manually segmenting the brains from the scans](#3.-manual-scan-segmentation)
1. [bulk-downloading atlas images](#4a.-downloading-atlas-images) and generating raster masks from them
1. [preprocessing scans before fine segmentation](#5.-preprocessing-scans)
1. [fine-segmenting brains into anatomical regions](#6.-fine-anatomical-segmentation)
1. [postprocessing segmentation results](#7.-aftermath)

## 1. Installations
This notebook can run in [GOOGLE COLAB](https://colab.research.google.com/github/zudanievil/MiceBrainMriSegmentation/blob/paper/instructions.ipynb) with no prior setup, but it is best to run it on a locally installed python.

To do this on a local machine:
1. clone this repository
1. install [Anaconda] package manager
1. [create virtual environment from the file] named `conda_env.yml` contained in this repository
1. install [Inkscape], install [ImageJ]
1. it is best to install [PyCharm] for code editing, and [set up PyCharm to use Anaconda virtual environment]

[Anaconda]: https://www.anaconda.com/products/individual
[create virtual environment from the file]: https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-from-an-environment-yml-file
[PyCharm]: https://www.jetbrains.com/pycharm/download/
[set up PyCharm to use Anaconda virtual environment]: https://www.jetbrains.com/help/pycharm/conda-support-creating-conda-virtual-environment.html
[Inkscape]: https://inkscape.org/
[ImageJ]: https://imagej.nih.gov/ij/download.html


In [24]:
YOUR_TEMP_DIRECTORY = "/home/admin/Documents/lab/example"

### utility functions

In [30]:
from urllib.request import urlretrieve
from zipfile import ZipFile
from shutil import copy as file_copy
# some utility functions

def unzip(src: Path, dst: Path):
    """unzip a `.zip file located at `src` into `dst` folder"""
    with zipfile.ZipFile(path_to_zip_file, 'r') as z:
        z.extractall(dst)
    
    
def rmdir_rec(folder: Path): #why didn't they implement it in the pathlib?
    """removes `folder` with all the contents"""
    for f in folder.iterdir():
        rmdir_rec(f) if f.is_dir() else f.unlink()
    folder.rmdir()
    
    
def merge_folders(src: Path, dst: Path, matching_files_policy: {"skip", "replace", "rename", "error"} = "replace"):
    """
    recursively copies contents from the `src` folder to `dst` folder.
    if files match, uses `matching_files_policy`:
        - skip
        - replace
        - error: raises FileExistsError
        - rename: adds ".copy" suffix to filename, eg `a.txt` -> `a.copy.txt`
    """
    assert matching_files_policy in {"skip", "replace", "rename", "error"}
    
    for path_src in src.iterdir():
        path_dst = dst / path_src.name
        if path_src.is_dir():
            path_dst.mkdir(exist_ok=True)
            merge_folders(path_src, path_dst, matching_files_policy)
        else:
            if path_dst.exists():
                if matching_files_policy == "error":
                    raise FileExistsError(path_dst)
                elif matching_files_policy == "replace":
                    path_dst.unlink()
                elif matching_files_policy == "rename":
                    while path_dst.exists():
                        path_dst = path_dst.parent / (path_dst.stem + ".copy" + path_dst.suffix)
                else:
                    continue
            file_copy(path_src, path_dst)

In [31]:
help(file_copy)

Help on function copy in module shutil:

copy(src, dst, *, follow_symlinks=True)
    Copy data and mode bits ("cp src dst"). Return the file's destination.
    
    The destination may be a directory.
    
    If follow_symlinks is false, symlinks won't be followed. This
    resembles GNU's "cp -P src dst".
    
    If source and destination are the same file, a SameFileError will be
    raised.



### setup

In [15]:
import sys
from pathlib import Path

RUNTIME_IS_ON_GOOGLE_COLAB: bool = "google.colab" in sys.modules

In [3]:
# if on google colab, fetch whole repository
# 
def fetch_lib_from_github() -> (str, "http message"):
    archive = Path("mbs_lib.zip")
    user = "zudanievil"
    repo = "MiceBrainMriSegmentation"
    branch = "paper"
    url = f"https://api.github.com/repos/{user}/{repo}/zipball/{branch}" 
    # https://docs.github.com/en/rest/reference/repos#download-a-repository-archive-tar
    dst_dir = "/content" # google VM directory for user contents
    
    zipball, msg = urlretrieve(url)
    unzip(src=Path(archive), dst=dst_dir)
    Path(archive).unlink()
    return f"{dst_dir}/{user}-{repo}", msg


if RUNTIME_IS_ON_GOOGLE_COLAB:
    MBS_LIB_PATH, msg = fetch_lib_from_github()
else:
    MBS_LIB_PATH = "./"

sys.path.insert(0, MBS_LIB_PATH)
from mbs_lib.core import info_classes as IC

## 2a. creating folders ##
classes `IC.ImageFolderInfo`, `IC.OntologyFolderInfo`, `IC.SegmentationResultsFolderInfo` contain `.write()` method that allows to conveniently create these folders in the filesystem

In [28]:
WORKING_DIR = "/content/example" if RUNTIME_IS_ON_GOOGLE_COLAB \
    else YOUR_TEMP_DIRECTORY


working_dir = pathlib.Path(WORKING_DIR)
working_dir.mkdir(exist_ok=False)

infos = {}

infos["ontology"] = IC.OntologyFolderInfo(working_dir/"ontology")
infos["ontology"].write()

for i in (1, 2):

    infos[f"images_{i}"] = IC.ImageFolderInfo(working_dir / f"images_{i}")
    infos[f"images_{i}"].write()

    infos[f"results_{i}"] = IC.SegmentationResultFolderInfo(
        infos[f"images_{i}"], infos["ontology"], folder=working_dir / f"results_{i}")
    infos[f"results_{i}"].write()

## 2b. putting files into image folder ##
* `lib/default_specifications` folder has configuration files that are copied by the `.write()` methods of info classes into the directory that they create. These are YAML files, that can be used to correct the `lib.pipelines` modules' behaviour.
* `image_folder_specification.yml` that was is created in `c:/users/user/example/img` must be edited.
* under the `file_name_fields` key there is a list of strings, that compose any file name that is related to image. one should give these fields names that correspond to current experiment parameters, like time, mice strain, etc.
    > e.g. if configuration looks like this:
    file_name_fields:
        - a
        - b
        - c
    then image file names are assumed to look like `{a}_{b}_{c}.npy`, and metadata names like `{a}_{b}_{c}.yml`. these names will be broken down by some scripts into "dictionaries" of {field_name: field_value}. For instance, name 10_20_xyz.npz will correspond to {"a":"10", "b":"20", "c":"xyz"}.  **It is possible to adjust cropped scan resolution based on the value of "frame" field through editing `cropped_image_shapes` in the image folder configuration.**
* After that transfer the images into the `img_raw` subfolder of image folder, rename them according to the fields.

In [ ]:
def fetch_example():
    
    

## 3. manual scan segmentation ##
* manual scan segmentation is done by ImageJ program.
* the result of the segmentation for each image is couple files that contain digits separated by tabulations and newlines (ImageJ scripts do not have sophisticated tools for file parsing and formatting). These files are collected by a python script into a single YAML metadata file, based on the values under `metadata keys` in the image folder configuration.
* to segment images do the following:
    1. make list of file names in `img_raw` subfolder, put it in the `ij_img_list.txt` file in the image folder, separated by newlines (only names, not the full paths). Create `ij_pointer.txt`, put 0 into it. Here is an easy way to o this:

In [ ]:
def list_raw_images():
    img_list_fname = image_f.folder()/"ij_img_list.txt"
    raw_img_folder = image_f.raw_image_folder()
    img_idx_fname = image_f.folder()/"ij_pointer.txt"

    with open(img_list_fname, "wt") as f:
        for img in raw_f.iterdir():
            print(img.name, file=f)
            
    with open(img_idx_fname, "wt") as f:
        print(0, file=f)
        
list_raw_images()
del list_raw_images

*   
   2. Open the `lib/imagej_scripts/brain_segmentation_with_imagej.txt`, edit command for image opening according to [ImageJ macro guide] and [ImageJ macro functions list] to open your images (the function below comment `//IMAGE OPENING FUNCTION`). Initially it opens 32bit little-endian single channel 256x256, written as a C-array.
   1. Open imagej, load that file as imagej macro: `macros > install > imagej`
   1. Press `h` to open log window with the instructuions. follow the instructions.
   1. Run collect `collect_image_metadata.main()` (cell below)
   >you can find about metadata collection process and how to tune it with configuration editing if you run `help(collect_image_metadata.main)`
   
[ImageJ macro guide]: https://imagej.nih.gov/ij/developer/macro/macros.html#tools
[ImageJ macro functions list]: https://imagej.nih.gov/ij/developer/macro/functions.html

**ADD TQDM PROGRESS BAR TO THIS FUNCTION**

In [ ]:
from lib.pipelines import collect_image_metadata
collect_image_metadata.main(image_f)

## 4a. downloading atlas images ##
variables for downloading are stored in `ontology_folder_specification.yml` file inside ontology folder that you've created above.

1. Edit `atlas_id` and `svg_groups` (a table of them can be found [here](http://help.brain-map.org/display/api/Atlas+Drawings+and+Ontologies))

In [ ]:
from lib.pipelines import download_svgs
download_svgs.download_default_ontology(ontology_f)

2. Edit the `slice_coordinates` if you know the coordinates for the first slice and the last slice in the atlas. this allows you to download a list of slice ids and zip it with coordinate range using a script in the cell below (the table is just for utility purposes, you can skip this step).

In [ ]:
download_svgs.download_slice_ids_table(ontology_f)

Now you have a very readable (for an xml) `default.xml` tree of structures for the atlas, and probably a tabulation-separated table of slice ids.
For each frame value in image names there should be an svg image with the same name. fill the ontology folder configuration  `svg_names_and_slice_ids` with pairs of `frame_field_value: slice_id`. run the script from the cell below to bulk-download the atlas vector images.

In [ ]:
download_svgs.download_svgs(ontology_f)

You can and need to manually edit the `.svg` masks with Inkscape, but note that some Inkscape functions can alter all xml attributes of the paths. structures for rendering are found by `structure_id` attributes of the paths, make sure, they are not deleted in the editing process.
1. I advise to compute mean of all images for each frame and put it into a corresponding svg (the rendering pipeline ignores visible objects that do not have `structure_id` attribute, so you can leave image there).
2. You **must** create a rectangle with xml attribute `structure_id='bbox'`, that will mark the border of the rendered masks (to edit xml attributes of objects from Inkscape, open the 'xml editor' tab.
3. It's a good idea to slightly adjust nodes of atlas structures to fit the contours of the brain.
4. Note that you can specify an arbitrary function to modify each mask right before the scan segmentation. Therefore you do need to flip/copy/crop the svg masks if only one hemisphere is covered and you need masks for both hemisperes, or otherwise. 

## 4b. rendering raster masks ##
For this one needs to edit `inkscape_executable_path` value in the ontology folder's configuration. 
The rendering pipeline uses image folder configuration `cropped_image_shapes` values for determining mask sizes.
Details about process can be found in `prerender_masks.main` function documentation. This process might take several hours, but is easily parallelized by calling pipeline from different processes for only a part of all frames.

In [ ]:
from lib.pipelines import prerender_masks
prerender_masks.main(ontology_f, image_f)

## 5. preprocessing scans ##
* Convert images to numpy native array format via `numpy.save()`, put them into `img` subfolder of the image folder.

* The pipeline called below uses `cropped_image_shapes` key of the configuration and `frame` field value in the image name as well as some metadata keys to crop and resize the images.

In [ ]:
from lib.pipelines import crop_images
crop_images.main(image_f)

To normalize image by intensity, we generate reference values and put them into metadata files. Later on you can choose which key to to use for image normalization. 

In [ ]:
from lib.pipelines import generate_intensity_reference
generate_intensity_reference.main(image_f)

## 6. fine anatomical segmentation
1. You need to specify a tabulation-separated table which contains names of images. Table header specifies, from which columns images fall into 'control' group and from which -- into the 'effect' group. images from the one row constitute a batch and are compared against each other. the `batch_for_comparison` pipeline can create this table. Note that one can easily edit this table in MS Excel/Libre Office, etc.
You can find about how to control pipeline behaviour through a result folder configuration in the `batch_for_comparison.main` description.

In [ ]:
from lib.pipelines import batch_for_comparison
batch_for_comparison.main(result_f)

2. please read the `compare_and_segment.main` help for detalis on how to control it's behaviour with configuration file and some other files, that can be created in the `spec` subfolder of results folder.

In [ ]:
from lib.pipelines import compare_and_segment
compare_and_segment.main(result_f)


3. segmentation results are saved into a temporary folder to save RAM and allow parallel execution (by running several processes and supplying `batch_range` argument to  `compare_and_segment.main`). Because of this design choice, segmentation results' pieces are aggregated by a separate function. You also need to delete temp folder yourself.

In [ ]:
compare_and_segment.collect_segmentation_results(result_f)

def cleanup():
    fs = result_f.segmentation_temp()
    [f.unlink for f in fs.iterdir()]
    fs.rmdir()
# cleanup()
del cleanup

## 7. aftermath
There are some scripts for processing the results table. Consult with their descriptions to understand what do they do.
also there is an execute_all script, that just calls them in a logical order (some scripts depend on each other). They also use configuration for the results folder.

In [ ]:
structures = (
    'Main Olfactory Bulb',
    'Olfactory Nerve',
    'Anterior Olfactory Nucleus',
    'Piriform Area',
    'Piriform-Amygdalar Area',
    'Cortical Amygdalar Area',
    'Entorhinal Area',
    'Olfactory Tubercle',
)

In [ ]:
segmentation_results_processing.execute_all(srfi, structure_list_for_significance_table=structures)